# IMU标定

## 导入依赖

In [1]:
import numpy as np
from imu_mpu6050 import RaspMPU6050
from sklearn.linear_model import LinearRegression

## 数据采集
从MPU6050读取原始数据

In [2]:
# 初始化MPU6050对象
imu = RaspMPU6050()

In [3]:
def sample_one_side(n_repeat = 10000):
    '''对IMU的单边采样'''
    data_raw = []
    for i in range(n_repeat):
        data_raw.append(imu.get_accel_raw())
    data_raw = np.array(data_raw)
    data_raw_mean = np.mean(data_raw, axis=0)
    return data_raw_mean

In [4]:
# 保持Z轴朝上
accel_z_up =  sample_one_side()
print(accel_z_up)

[ 152.7002  135.1152 8797.2304]


In [5]:
# 保持Z轴朝下
accel_z_down = sample_one_side()
print(accel_z_down)

[ -148.1456   -34.4474 -7746.1666]


In [6]:
# 保持X轴朝上
accel_x_up = sample_one_side()
print(accel_x_up)

[8261.148    12.5242  690.1688]


In [7]:
# 保持X轴朝下
accel_x_down = sample_one_side()
print(accel_x_down)

[-8088.8126   -67.1232   250.5438]


In [11]:
# 保持Y轴朝上
accel_y_up = sample_one_side()
print(accel_y_up)

[ 666.1468 8364.7666  608.4802]


In [13]:
# 保持Y轴朝下
accel_y_down = sample_one_side()
print(accel_y_down)

[  224.0862 -8102.1206   699.5064]


## 制作数据集

In [14]:
# 当地的重力加速度 m/s^2
g = 9.7773

# # 重力加速度 六面原始数据
# accel_raw_data = np.array([
#     [2017, 138, 15286], # Z轴朝上
#     [2399, -568, -18220], # Z轴朝下
#     [18475, -234, -1084], # X轴朝上
#     [-14474, -206.62, -2367], # X轴朝下
#     [2307, 16132, -2579], # Y轴朝上
#     [2057, -16523, -545], # Y轴朝下
# ])

# 重力加速度 六面原始数据
accel_raw_data = np.array([
    list(accel_z_up), # Z轴朝上
    list(accel_z_down), # Z轴朝下
    list(accel_x_up), # X轴朝上
    list(accel_x_down), # X轴朝下
    list(accel_y_up), # Y轴朝上
    list(accel_y_down), # Y轴朝下
])

# 重力加速度 六面目标数据
# 注:加速度计采样的加速度方向跟真实的重力加速度方向相反
accel_target = np.array([
    [0, 0, g], # Z轴朝上
    [0, 0, -g], # Z轴朝下
    [g, 0, 0], # X轴朝上
    [-g, 0, 0],  # X轴朝下
    [0, g, 0], # Y轴朝上
    [0, -g, 0], # Y轴朝下
])

## 线性最小二乘法

使用sklearn进行线性最小二乘法求解

标定加速度的X轴

In [15]:
ax_input = accel_raw_data[:, 0].reshape((-1, 1))
ax_output = accel_target[:, 0].reshape((-1, 1))
regx = LinearRegression().fit(ax_input, ax_output)

ax_scale, ax_bias = regx.coef_[0][0] , regx.intercept_ [0]
print('ax_real = {} * ax_raw + {}'.format(ax_scale, ax_bias))

ax_real = 0.001192752067567948 * ax_raw + -0.21213552743321828


标定加速度的y轴

In [16]:
ay_input = accel_raw_data[:, 1].reshape((-1, 1))
ay_output = accel_target[:, 1].reshape((-1, 1))
regy = LinearRegression().fit(ay_input, ay_output)

ay_scale, ay_bias = regy.coef_[0][0] , regy.intercept_ [0]
print('ay_real = {} * ay_raw + {}'.format(ay_scale, ay_bias))

ay_real = 0.0011871364200602235 * ay_raw + -0.06108109708193459


标定Z轴的数据

In [17]:
az_input = accel_raw_data[:, 2].reshape((-1, 1))
az_output = accel_target[:, 2].reshape((-1, 1))
regz = LinearRegression().fit(az_input, az_output)

az_scale, az_bias = regz.coef_[0][0] , regz.intercept_ [0]
print('az_real = {} * az_raw + {}'.format(az_scale, az_bias))

az_real = 0.001180842335279502 * az_raw + -0.6494166411314827


## 保存标定数据

In [18]:
import json

In [19]:
with open('config/imu_calibration.json', 'w') as f:
    cali_data_dict = {}
    cali_data_dict['ax_scale'] = float(ax_scale)
    cali_data_dict['ay_scale'] = float(ay_scale)
    cali_data_dict['az_scale'] = float(az_scale)
    cali_data_dict['ax_bias'] = float(ax_bias)
    cali_data_dict['ay_bias'] = float(ay_bias)
    cali_data_dict['az_bias'] = float(az_bias)
    print(json.dumps(cali_data_dict))
    f.write(json.dumps(cali_data_dict))

{"ax_scale": 0.001192752067567948, "ay_scale": 0.0011871364200602235, "az_scale": 0.001180842335279502, "ax_bias": -0.21213552743321828, "ay_bias": -0.06108109708193459, "az_bias": -0.6494166411314827}


载入标定数据演示

In [20]:
with open('config/imu_calibration.json', 'r') as f:
    data = json.loads(f.read())
    print(data)
    print(data['ax_scale'])

{'ax_scale': 0.001192752067567948, 'ay_scale': 0.0011871364200602235, 'az_scale': 0.001180842335279502, 'ax_bias': -0.21213552743321828, 'ay_bias': -0.06108109708193459, 'az_bias': -0.6494166411314827}
0.001192752067567948
